In [76]:
from typing import Tuple

#from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
torch.manual_seed(99)

from matplotlib import pyplot as plot

In [77]:
df = pd.read_csv("./data/datasets/historical.data")
#df = df.set_index("date").sort_index(ascending=True)
df = df.sort_values(by="date",ascending=True)
df["month"] = df.apply(lambda x: int(x["date"].split("-")[1])-1, axis=1) # category indexed to 0
df["day"] = df.apply(lambda x: int(x["date"].split(" ")[0].split("-")[2])-1, axis=1) # category indexed to 0
df["hour"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[0]), axis=1)
df["minute"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[1]), axis=1)
df = df.drop(["date"], axis=1)
#df.apply(lambda x: x["date"].split(" ")[0].split("-")[1] + "-" + x["date"].split(" ")[0].split("-")[2], axis=1)
#display(df)
df_inside = df.loc[df["label"] == "inside"].drop(["label"], axis=1)
df_outside = df.loc[df["label"] == "outside"].drop(["label"], axis=1)
print("inside")
display(df_inside)
#print("outside")
#display(df_outside)

inside


,temperature,month,day,hour,minute
1,22.875,1,21,0,30
2,22.937,1,21,1,0
5,23.062,1,21,1,30
7,23.125,1,21,2,0
8,23.187,1,21,2,30
...,...,...,...,...,...
97237,24.812,10,29,21,30
97239,24.750,10,29,22,0
97241,24.500,10,29,22,30
97242,24.437,10,29,23,0


In [78]:



inside_arr=np.array(df_inside)
outside_arr=np.array(df_outside)

#print(inside_arr)

#train_test_split(inside_arr, shuffle=False)

def split_data(arr: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    total_len,_=arr.shape

    train_p = 0.80
    test_p = 0.10
    ivs_p = 0.10

    train_len = round(total_len*train_p)
    test_len = round(total_len*test_p)
    ivs_len = round(total_len*ivs_p)

    model_len = train_len+test_len

    total_used_len = model_len+ivs_len

    train = arr[:train_len]
    test = arr[train_len:model_len]
    ivs = arr[model_len:total_used_len]

    return train, test, ivs


inside_train, inside_test, inside_ivs = split_data(inside_arr)

print("train")
print(inside_train)
print("test")
print(inside_test)
print("ivs")
print(inside_ivs)

train
[[22.875  1.    21.     0.    30.   ]
 [22.937  1.    21.     1.     0.   ]
 [23.062  1.    21.     1.    30.   ]
 ...
 [23.562  4.    11.     8.     0.   ]
 [23.437  4.    11.     8.    30.   ]
 [23.562  4.    11.     9.     0.   ]]
test
[[23.625  4.    11.     9.    30.   ]
 [23.437  4.    11.    10.     0.   ]
 [23.562  4.    11.    10.    30.   ]
 ...
 [24.     7.    20.    15.     0.   ]
 [24.312  7.    20.    15.    30.   ]
 [24.625  7.    20.    16.     0.   ]]
ivs
[[24.375  7.    20.    16.    30.   ]
 [24.625  7.    20.    17.     0.   ]
 [24.562  7.    20.    17.    30.   ]
 ...
 [24.75  10.    29.    22.     0.   ]
 [24.5   10.    29.    22.    30.   ]
 [24.437 10.    29.    23.     0.   ]]


In [79]:
# 2h30

per_min = (2*60) + 30
per_30_mins = per_min / 30

print("Number of Wanted Predictions: " + str(per_30_mins))

Number of Wanted Predictions: 5.0


One-step-ahead (auto-regressive)
<br>
One-step-ahead (auto-regressive with exogenous inputs)
<br>
Recursive
<br>
MIMO
<br>
Direct
<br>
Hybrid (Recursive and Direct)
<br>
Hybrid (Direct and MIMO)
<br>
<br>
MIMO is good too, but it would be more accurate and faster to compute several dedicated models for each step instead of all in a batch.
<br>
Direct and Hybrid(Direct and MIMO) are on top - they can execute all
<br>
<br>
Hybrid(Direct and MIMO) = [2,2,1] predictive models
<br>
Direct = [+-,1,1,1,1] predictive models
<br>
<br>
Other tests that could be implemented would be creating a Long Short-Term Memory model, trying out some kind of BERT model with months as categorial variables, or classical statistic models like SVMs or Decision Trees.

In [80]:
inside_train_0 = []
inside_train_X_0 = []
inside_train_Y_0 = []

imput_dim_df = 48 # 24H input

_temp_arr = []
_n = 0
for temp, month, day, hour, minute in inside_train:
    if len(_temp_arr) < imput_dim_df:
        _temp_arr.append(temp)
        _n+=1
        continue
    else:
        _temp_arr.pop(0)
        _temp_arr.append(temp)

    if _n < len(inside_train)-1:
        inside_train_X_0.append(_temp_arr)
        inside_train_Y_0.append(inside_train[_n+1][0])
        
        inside_train_0.append((torch.Tensor(_temp_arr), inside_train[_n+1][0]))

    _n+=1

inside_train_X_0 = np.array(inside_train_X_0, dtype=np.float32)
inside_train_Y_0 = np.array(inside_train_Y_0, dtype=np.float32)
#inside_train_X_0 = torch.from_numpy(np.array(inside_train_X_0, dtype=np.float32))
#inside_train_Y_0 = torch.from_numpy(np.array(inside_train_Y_0, dtype=np.float32))


In [81]:
inside_test_0 = []
inside_test_X_0 = []
inside_test_Y_0 = []

imput_dim_df = 48 # 24H input

_temp_arr = []
_n = 0
for temp, month, day, hour, minute in inside_test:
    if len(_temp_arr) < imput_dim_df:
        _temp_arr.append(temp)
        _n+=1
        continue
    else:
        _temp_arr.pop(0)
        _temp_arr.append(temp)

    if _n < len(inside_test)-1:
        inside_test_X_0.append(_temp_arr)
        inside_test_Y_0.append(inside_test[_n+1][0])
        
        inside_test_0.append((torch.Tensor(_temp_arr), inside_test[_n+1][0]))

    _n+=1

inside_test_X_0 = np.array(inside_test_X_0, dtype=np.float32)
inside_test_Y_0 = np.array(inside_test_Y_0, dtype=np.float32)
#inside_test_X_0 = torch.from_numpy(np.array(inside_test_X_0, dtype=np.float36))
#inside_test_Y_0 = torch.from_numpy(np.array(inside_test_Y_0, dtype=np.float36))


In [82]:
#print(inside_train_X_0.size())
#print(inside_train_Y_0.size())
print(len(inside_train_X_0))
print(len(inside_test_X_0))

38849
4813


In [84]:
_ = len(inside_train_X_0) / 50
print(len(inside_train_X_0) / 50)
print(int(_) * 50)

776.98
38800


# BREAK

In [85]:
train_data = inside_train_Y_0[0:38649]
test_data = inside_test_Y_0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [86]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
#train_data_normalized = train_data

In [87]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1).to(device)

In [88]:
train_window = 50 # 48H + margin input
#train_window = 12

In [89]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [90]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [91]:
train_inout_seq[:5]

[(tensor([ 0.3542,  0.3698,  0.3542,  0.3698,  0.3698,  0.3226,  0.2753,  0.2596,
           0.2596,  0.2437,  0.1966,  0.1966,  0.1651,  0.1651,  0.1492,  0.1336,
           0.1492,  0.1336,  0.1177,  0.1021,  0.0862,  0.0706,  0.0706,  0.0390,
           0.0232,  0.0075,  0.0232,  0.0232, -0.0083, -0.0240, -0.0240, -0.0555,
          -0.0555, -0.0240, -0.0083,  0.0075,  0.0547,  0.1177,  0.1966,  0.2281,
           0.1807,  0.2281,  0.2596,  0.3226,  0.3068,  0.2753,  0.3542,  0.3542,
           0.3226,  0.3383], device='cuda:0'),
  tensor([0.3383], device='cuda:0')),
 (tensor([ 0.3698,  0.3542,  0.3698,  0.3698,  0.3226,  0.2753,  0.2596,  0.2596,
           0.2437,  0.1966,  0.1966,  0.1651,  0.1651,  0.1492,  0.1336,  0.1492,
           0.1336,  0.1177,  0.1021,  0.0862,  0.0706,  0.0706,  0.0390,  0.0232,
           0.0075,  0.0232,  0.0232, -0.0083, -0.0240, -0.0240, -0.0555, -0.0555,
          -0.0240, -0.0083,  0.0075,  0.0547,  0.1177,  0.1966,  0.2281,  0.1807,
           0.

In [92]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)


        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [93]:
def load_model():
    model = LSTM().to(device)
    loss_function = nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    return model, loss_function, optimizer
model, loss_function, optimizer = load_model()

In [94]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [96]:
epochs = 25
model, loss_function, optimizer = load_model()

for i in range(epochs):
    _ix = -1
    for seq, labels in train_inout_seq:
        seq = seq.to(device)
        labels = labels.to(device)

        _ix +=1
        
        if torch.isnan(seq).any().item():
            print(f"nan values in seq at {_ix}")
            continue

        if pd.isna(labels.item()):
            print(f"nan labels at {_ix}")
            continue

        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)
        if pd.isna(y_pred.item()):
            print(f"nan preds at {_ix}")
            continue

        single_loss = loss_function(y_pred, labels)
        if pd.isna(single_loss.item()):
            print(f"nan loss at {_ix}")
            raise
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.00002032
epoch:  24 loss: 0.0001778315


In [97]:
fut_pred = 12

test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)

[0.5903587341308594, 0.5432186126708984, 0.5588479042053223, 0.5273370742797852, 0.4486861228942871, 0.49582624435424805, 0.49582624435424805, 0.4801969528198242, 0.4486861228942871, 0.4486861228942871, 0.46431541442871094, 0.46431541442871094, 0.43280458450317383, 0.3856644630432129, 0.4012937545776367, 0.41717529296875, 0.4012937545776367, 0.3856644630432129, 0.43280458450317383, 0.4012937545776367, 0.43280458450317383, 0.4486861228942871, 0.5432186126708984, 0.5588479042053223, 0.4486861228942871, 0.43280458450317383, 0.46431541442871094, 0.5432186126708984, 0.6062402725219727, 0.5903587341308594, 0.6062402725219727, 0.5432186126708984, 0.6692619323730469, 0.5432186126708984, 0.5432186126708984, 0.5273370742797852, 0.6377511024475098, 0.6218695640563965, 0.6692619323730469, 0.6533803939819336, 0.6533803939819336, 0.6377511024475098, 0.6218695640563965, 0.6218695640563965, 0.6377511024475098, 0.6062402725219727, 0.5903587341308594, 0.5747294425964355, 0.5903587341308594, 0.5747294425

In [98]:
model.eval()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:]).to(device)
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

In [99]:
test_inputs[fut_pred:]

[0.43280458450317383,
 0.3856644630432129,
 0.4012937545776367,
 0.41717529296875,
 0.4012937545776367,
 0.3856644630432129,
 0.43280458450317383,
 0.4012937545776367,
 0.43280458450317383,
 0.4486861228942871,
 0.5432186126708984,
 0.5588479042053223,
 0.4486861228942871,
 0.43280458450317383,
 0.46431541442871094,
 0.5432186126708984,
 0.6062402725219727,
 0.5903587341308594,
 0.6062402725219727,
 0.5432186126708984,
 0.6692619323730469,
 0.5432186126708984,
 0.5432186126708984,
 0.5273370742797852,
 0.6377511024475098,
 0.6218695640563965,
 0.6692619323730469,
 0.6533803939819336,
 0.6533803939819336,
 0.6377511024475098,
 0.6218695640563965,
 0.6218695640563965,
 0.6377511024475098,
 0.6062402725219727,
 0.5903587341308594,
 0.5747294425964355,
 0.5903587341308594,
 0.5747294425964355,
 0.5856366753578186,
 0.5953073501586914,
 0.6032483577728271,
 0.6102455854415894,
 0.6169114112854004,
 0.6236931681632996,
 0.6307205557823181,
 0.6380606293678284,
 0.6456950306892395,
 0.6535345

In [101]:
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)

[[24.29326766]
 [24.33163014]
 [24.36313123]
 [24.39088845]
 [24.41733104]
 [24.44423351]
 [24.47211037]
 [24.50122762]
 [24.53151244]
 [24.56261086]
 [24.59400319]
 [24.62503446]]


In [115]:

#scaler = MinMaxScaler(feature_range=(-1, 1))
#train_data_normalized = scaler.fit_transform(inside_test_X_0.reshape(-1, 1))
#
#fut_pred = 12
#
#test_inputs = train_data_normalized[-train_window:].tolist()
#print(test_inputs)
#
#model.eval()
#
#for i in range(fut_pred):
#    seq = torch.FloatTensor(test_inputs[-train_window:]).to(device)
#    with torch.no_grad():
#        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
#                        torch.zeros(1, 1, model.hidden_layer_size))
#        test_inputs.append(model(seq).item())

def predict(input):
    X = scaler.fit_transform(input.reshape(-1, 1))
    X = seq = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        preds = model(X)
        Y = np.array(preds.cpu())
    actual_predictions = scaler.inverse_transform(Y.reshape(-1, 1))
    return actual_predictions

print(predict(inside_test_X_0[0:49]))
print(inside_test_Y_0[50])

[[24.727251]]
23.5


In [ ]:
class LSTM(nn.Module):

    hidden_layer_size: int
    lstm: nn.LSTM
    linear: nn.Linear
    hidden_cell: tuple[torch.Tensor, torch.Tensor]
    scaler: MinMaxScaler
    device: str

    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)


        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

        self.scaler = MinMaxScaler(feature_range=(-1, 1))

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(self.device)

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def predict(self, input):
        X = scaler.fit_transform(input.reshape(-1, 1))

    def train(self, train_data, train_window = 50):

        # Initiate loss function and optimizer

        loss_function = nn.MSELoss().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Scale
        
        train_data_normalized = self.scaler.fit_transform(train_data.reshape(-1, 1))
        train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1).to(self.device)

        # Create sequences
        
        train_inout_seq = []
        L = len(train_data_normalized)
        for i in range(L-train_window):
            train_seq = train_data_normalized[i:i+train_window]
            train_label = train_data_normalized[i+train_window:i+train_window+1]
            train_inout_seq.append((train_seq ,train_label))
        
        # Train the model

        for i in range(epochs):
            _ix = -1
            for seq, labels in train_inout_seq:
                seq = seq.to(self.device)
                labels = labels.to(self.device)

                _ix +=1
                
                if torch.isnan(seq).any().item():
                    print(f"nan values in seq at {_ix}")
                    continue

                if pd.isna(labels.item()):
                    print(f"nan labels at {_ix}")
                    continue

                optimizer.zero_grad()
                self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(device),
                                torch.zeros(1, 1, self.hidden_layer_size).to(device))

                y_pred = self(seq)
                if pd.isna(y_pred.item()):
                    print(f"nan preds at {_ix}")
                    continue

                single_loss = loss_function(y_pred, labels)
                if pd.isna(single_loss.item()):
                    print(f"nan loss at {_ix}")
                    raise
                single_loss.backward()
                optimizer.step()

            if i%25 == 1:
                print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

        print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

        # Finalize
        self.eval()

In [ ]:
def train(train_data, train_window = 50):

    # Initiate loss function and optimizer

    loss_function = nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Scale
    
    train_data_normalized = self.scaler.fit_transform(train_data.reshape(-1, 1))
    train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1).to(self.device)

    # Create sequences
    
    train_inout_seq = []
    L = len(train_data_normalized)
    for i in range(L-train_window):
        train_seq = train_data_normalized[i:i+train_window]
        train_label = train_data_normalized[i+train_window:i+train_window+1]
        train_inout_seq.append((train_seq ,train_label))
    
    # Train the model

    for i in range(epochs):
        _ix = -1
        for seq, labels in train_inout_seq:
            seq = seq.to(self.device)
            labels = labels.to(self.device)

            _ix +=1
            
            if torch.isnan(seq).any().item():
                print(f"nan values in seq at {_ix}")
                continue

            if pd.isna(labels.item()):
                print(f"nan labels at {_ix}")
                continue

            optimizer.zero_grad()
            self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(device))

            y_pred = self(seq)
            if pd.isna(y_pred.item()):
                print(f"nan preds at {_ix}")
                continue

            single_loss = loss_function(y_pred, labels)
            if pd.isna(single_loss.item()):
                print(f"nan loss at {_ix}")
                raise
            single_loss.backward()
            optimizer.step()

        if i%25 == 1:
            print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

    # Finalize
    self.eval()

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq


######

epochs = 25
model, loss_function, optimizer = load_model()

for i in range(epochs):
    _ix = -1
    for seq, labels in train_inout_seq:
        seq = seq.to(device)
        labels = labels.to(device)

        _ix +=1
        
        if torch.isnan(seq).any().item():
            print(f"nan values in seq at {_ix}")
            continue

        if pd.isna(labels.item()):
            print(f"nan labels at {_ix}")
            continue

        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)
        if pd.isna(y_pred.item()):
            print(f"nan preds at {_ix}")
            continue

        single_loss = loss_function(y_pred, labels)
        if pd.isna(single_loss.item()):
            print(f"nan loss at {_ix}")
            raise
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.00002032
epoch:  24 loss: 0.0001778315
